In [ ]:
import chess.pgn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Parse PGN file and extract moves
def parse_pgn(pgn_file):
    games = []
    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            moves = []
            node = game
            while not node.is_end():
                next_node = node.variation(0)
                move = node.board().san(next_node.move)
                moves.append(move)
                node = next_node
            games.append(moves)
    return games

games = parse_pgn('Carlsen.pgn')

# Convert board to matrix
def board_to_matrix(board):
    piece_map = board.piece_map()
    matrix = np.zeros((8, 8), dtype=np.int8)
    for square, piece in piece_map.items():
        matrix[square // 8, square % 8] = piece.piece_type * (1 if piece.color == chess.WHITE else -1)
    return matrix

# Generate input/output pairs
X, y = [], []
for moves in games:
    board = chess.Board()
    for move in moves:
        X.append(board_to_matrix(board))
        board.push_san(move)
        y.append(board_to_matrix(board))

X = np.array(X)
y = np.array(y)

In [ ]:
# Reshape for CNN input
X = X.reshape(-1, 8, 8, 1)
y = y.reshape(-1, 8, 8, 1)

# One-hot encode y for 13 possible pieces (6 types * 2 colors + empty)
def to_one_hot(y):
    # Map: 0=empty, 1-6=white, -1 to -6=black
    one_hot = np.zeros((y.shape[0], 8, 8, 13), dtype=np.int8)
    for idx, board in enumerate(y):
        for i in range(8):
            for j in range(8):
                v = board[i, j, 0]
                if v == 0:
                    one_hot[idx, i, j, 0] = 1
                elif v > 0:
                    one_hot[idx, i, j, v] = 1
                else:
                    one_hot[idx, i, j, 6 - v] = 1  # -1 to -6 maps to 7-12
    return one_hot

y_onehot = to_one_hot(y)

# Split into train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y_onehot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input

model = Sequential([
    Input(shape=(8, 8, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(8 * 8 * 13, activation='softmax'),
    Reshape((8, 8, 13))
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32
)

Epoch 1/5
12173/12173 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - accuracy: 0.9484 - loss: 0.2027 - val_accuracy: 0.9491 - val_loss: 0.2028
Epoch 2/5
12173/12173 ━━━━━━━━━━━━━━━━━━━━ 198s 16ms/step - accuracy: 0.9511 - loss: 0.1930 - val_accuracy: 0.9502 - val_loss: 0.1974
Epoch 3/5
12173/12173 ━━━━━━━━━━━━━━━━━━━━ 199s 16ms/step - accuracy: 0.9528 - loss: 0.1869 - val_accuracy: 0.9518 - val_loss: 0.1934
Epoch 4/5
12173/12173 ━━━━━━━━━━━━━━━━━━━━ 199s 16ms/step - accuracy: 0.9540 - loss: 0.1826 - val_accuracy: 0.9538 - val_loss: 0.1880
Epoch 5/5
12173/12173 ━━━━━━━━━━━━━━━━━━━━ 199s 16ms/step - accuracy: 0.9549 - loss: 0.1790 - val_accuracy: 0.9530 - val_loss: 0.1902


In [ ]:
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print(f"Train accuracy: {train_acc:.4f}")
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

model.save('chess_cnn_model.keras')

Train accuracy: 0.9552
Validation accuracy: 0.9530
Test accuracy: 0.9529
